# Linking loop data

In [20]:
import os
import shutil
import glob
import pandas as pd

In [21]:
project_dir = '<project-dir>'
lji_lcsd_hub = 'results/lji_lcsd_hub/release-0.1/hub/'
os.chdir(project_dir)

## Making a shortcut for FitHiChIP loops (All types)

Include all loop types

In [ ]:
glob_path = 'results/loops/fithichip/*_*/*/FitHiChIP_Peak2ALL_b*_L20000_U2000000/'
glob_path += 'P2PBckgr_*/Coverage_Bias/FitHiC_BiasCorr/FitHiChIP-*.interactions_FitHiC_Q0.01.bed'
fns = glob.glob(glob_path)

In [34]:
# drop hichip-peaks.peaks, these will not be on the LJI-LCSD
fns = [x for x in fns if 'hichip-peaks.peaks' not in x]

In [36]:
for fn in fns:
    
    # get sample information
    file_info = fn.split('/')
    sample_name, peak_type = file_info[3].rsplit('_', 1)
    stringency = file_info[4][0]
    resolution = file_info[4][1:] + '000'
    stringency_long = 'stringent' if stringency == 'S' else 'loose'
    
    # determine the reference genome
    if 'Homo_Sapien' in sample_name:
        ref = 'hg38'
    elif 'Mus_Musculus' in sample_name:
        ref = 'mm10'
    
    # determine the shortcut directory
    if peak_type == 'chipseq.peaks':
        sample_outdir = '{ref}/loops/hichip/chip-seq/macs2/{stringeny}'

    elif peak_type == 'hichip-peaks.peaks':
        sample_outdir = '{ref}/loops/hichip/hichip/hichip-peaks/{stringeny}'
        
    elif peak_type == 'fithichip.peaks':
        sample_outdir = '{ref}/loops/hichip/hichip/fithichip-utility/{stringeny}'
        
    sample_outdir = sample_outdir.format(ref=ref, stringeny=stringency_long)
        
    # get the link name
    new_fn = '{sample_name}.{res}.fithichip_q0.01.loops.bed'
    new_fn = new_fn.format(sample_name=sample_name, res=resolution)
    new_path = os.path.join(project_dir, lji_lcsd_hub, sample_outdir, new_fn) 
    
    # create the link
    if not os.path.islink(new_path):
        
        print(new_path)
        abs_fn = os.path.join(project_dir, fn)
        os.symlink(abs_fn, new_path)


#### Account of loop types

In [37]:
acct_data = []
for fn in fns:
    
    # get sample information
    file_info = fn.split('/')
    sample_name, peak_type = file_info[3].rsplit('_', 1)
    stringency = file_info[4][0]
    resolution = file_info[4][1:] + '000'
    stringency_long = 'stringent' if stringency == 'S' else 'loose'
    
    # determine the reference genome
    if 'Homo_Sapien' in sample_name:
        ref = 'hg38'
    elif 'Mus_Musculus' in sample_name:
        ref = 'mm10'
        
    acct_data.append([sample_name, peak_type, stringency, resolution, ref])
    

In [38]:
acct_df = pd.DataFrame(acct_data)
acct_df.columns = ['sample_name', 'peak_type', 'stringency', 'resolution', 'ref']
acct_agg = acct_df.groupby(['ref', 'peak_type', 'stringency', 'resolution'])
acct_agg.count()

sample_name
ref  peak_type       stringency resolution             
hg38 chipseq.peaks   L          10000               240
                                25000               240
                                5000                238
                     S          10000               240
                                25000               240
                                5000                240
     fithichip.peaks L          10000               756
                                25000               756
                                5000                756
                     S          10000               756
                                25000               756
                                5000                756
mm10 chipseq.peaks   L          10000                75
                                25000                75
                                5000                 75
                     S          10000                75
                                25000                75
                                5000                 75
     fithichip.peaks L          10000               281
                                25000               281
                                5000                281
                     S          10000               281
                                25000               281
                                5000                281

## Making a shortcut for HiCCUPs loops

In [16]:
glob_path = 'results/loops/hiccups/whole_genome/*/postprocessed_pixels_*.bedpe'
fns = glob.glob(glob_path)

In [19]:
len(fns)

528

In [17]:
for fn in fns:
    
    # get sample information
    file_info = fn.split('/')
    sample_name = file_info[4]
    resolution = file_info[5].split('_')[2].split('.')[0]
    
    # determine the reference genome
    if 'Homo_Sapien' in sample_name:
        ref = 'hg38'
    elif 'Mus_Musculus' in sample_name:
        ref = 'mm10'
    
    # determine the shortcut directory
    sample_outdir = '{ref}/loops/hichip/hiccups/'.format(ref=ref)
        
    # get the link name
    new_fn = '{sample_name}.{res}.post_processed.hiccups.loops.bed'
    new_fn = new_fn.format(sample_name=sample_name, res=resolution)

    new_path = os.path.join(project_dir, lji_lcsd_hub, sample_outdir, new_fn) 
    
    if not os.path.islink(new_path):
        abs_fn = os.path.join(project_dir, fn)
        os.symlink(abs_fn, new_path)
    